<a href="https://colab.research.google.com/github/jsilbergDS/CS236project/blob/main/EndOfRoad_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adapted from: https://github.com/JiahuiYu/generative_inpainting



In [ ]:
!pip install git+https://github.com/JiahuiYu/neuralgym
!git clone https://github.com/JiahuiYu/generative_inpainting.git

  Cloning https://github.com/JiahuiYu/neuralgym to /tmp/pip-req-build-7o5_gr9d
  Running command git clone -q https://github.com/JiahuiYu/neuralgym /tmp/pip-req-build-7o5_gr9d
  Created wheel for neuralgym: filename=neuralgym-0.0.1-py3-none-any.whl size=40441 sha256=8f7a986ebabf4b1f8c60ea3bd0c784e43df4547484f95b517cfbbdeda4440880
  Stored in directory: /tmp/pip-ephem-wheel-cache-52b7k2eg/wheels/bf/3a/34/026a730c199a8c4cb5da3c052c71801eaf6618c4771d7faab2
Successfully built neuralgym


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import os
training_file_names = []
validation_file_names = []
overall_dir = '/content/drive/MyDrive/BayArea500NoHighways/train'
file_list = [f for f in os.listdir(overall_dir) if not f.startswith('.')] 
for file in file_list:
  training_file_names.append(overall_dir+file)
overall_dir = '/content/drive/MyDrive/BayArea500NoHighways/train'
file_list = [f for f in os.listdir(overall_dir) if not f.startswith('.')] 
for file in file_list:
  validation_file_names.append(overall_dir+file)


with open("train_shuffled.flist", "w") as train_out:
  train_out.write("\n".join(training_file_names))

with open("validation.flist", "w") as val_out:
  val_out.write("\n".join(validation_file_names))
  


I modify inpaint.yml to load the places2 model as a starting point



```
model_restore: '/content/drive/MyDrive/generative_inpainting/logs/places2' 
```



In [ ]:
!python /content/drive/MyDrive/generative_inpainting/train.py

For the pix2pix masks I replaced the following code in inpaint_ops:

Replace: 



```
 def generate_mask(H, W):
        average_radius = math.sqrt(H*H+W*W) / 8
        mask = Image.new('L', (W, H), 0)

        for _ in range(np.random.randint(1, 4)):
            num_vertex = np.random.randint(min_num_vertex, max_num_vertex)
            angle_min = mean_angle - np.random.uniform(0, angle_range)
            angle_max = mean_angle + np.random.uniform(0, angle_range)
            angles = []
            vertex = []
            for i in range(num_vertex):
                if i % 2 == 0:
                    angles.append(2*math.pi - np.random.uniform(angle_min, angle_max))
                else:
                    angles.append(np.random.uniform(angle_min, angle_max))

            h, w = mask.size
            vertex.append((int(np.random.randint(0, w)), int(np.random.randint(0, h))))
            for i in range(num_vertex):
                r = np.clip(
                    np.random.normal(loc=average_radius, scale=average_radius//2),
                    0, 2*average_radius)
                new_x = np.clip(vertex[-1][0] + r * math.cos(angles[i]), 0, w)
                new_y = np.clip(vertex[-1][1] + r * math.sin(angles[i]), 0, h)
                vertex.append((int(new_x), int(new_y)))

            draw = ImageDraw.Draw(mask)
            width = int(np.random.uniform(min_width, max_width))
            draw.line(vertex, fill=1, width=width)
            for v in vertex:
                draw.ellipse((v[0] - width//2,
                              v[1] - width//2,
                              v[0] + width//2,
                              v[1] + width//2),
                             fill=1)

        if np.random.normal() > 0:
            mask.transpose(Image.FLIP_LEFT_RIGHT)
        if np.random.normal() > 0:
            mask.transpose(Image.FLIP_TOP_BOTTOM)
        mask = np.asarray(mask, np.float32)
        mask = np.reshape(mask, (1, H, W, 1))
        return mask
```

With: 

```
    def generate_mask(H, W):
        new_rand = np.random.randint(1,250)
        sec_rand = np.random.randint(0,7)
        mask = Image.open("/content/drive/MyDrive/fixed_masks/output_"+str(new_rand)+"_fake_B_"+str(sec_rand)+".png")
        print(new_rand," ",sec_rand)
        if np.random.normal() > 0:
            mask.transpose(Image.FLIP_LEFT_RIGHT)
        if np.random.normal() > 0:
            mask.transpose(Image.FLIP_TOP_BOTTOM)
        mask = np.asarray(mask, np.float32)
        mask = mask[:,:,0]
        mask = np.reshape(mask, (1, H, W, 1))
        return mask

```
```

### Image generation ###

I modify below to generate all test set outputs. Bay_only is the fine-tuned model. 

In [ ]:
%cd /content/drive/MyDrive/generative_inpainting
!python /content/drive/MyDrive/generative_inpainting/test.py --image /content/drive/MyDrive/NOLATest/test/output_5.png --mask /content/drive/MyDrive/test_masks/output_5_fake_B_0.png --output /content/drive/MyDrive/NOLATest/test/output_5_gen_final.png --checkpoint_dir /content/drive/MyDrive/generative_inpainting/logs/bay_only


### Metrics ###

The following section prepares the files to call batch_test.py on my 54 validation image / generated image **pairs**

In [ ]:
import os
LPIPS_file_names = []
overall_dir = '/content/drive/MyDrive/BayArea500NoHighways/val/'
file_list = [f for f in os.listdir(overall_dir) if not f.startswith('.')] 
i = 0
for file in file_list:
  LPIPS_file_names.append(overall_dir+file+" "+"/content/drive/MyDrive/fixed_masks/output_"+str(i)+"_fake_B_0.png"+" "+"/content/drive/MyDrive/BayArea500NoHighways/metrics/out_"+file)
  i+=1
with open("/content/test.flist", "w") as val_out:
  val_out.write("\n".join(LPIPS_file_names))

['/content/drive/MyDrive/BayArea500NoHighways/val/output_451.png /content/drive/MyDrive/fixed_masks/output_0_fake_B_0.png /content/drive/MyDrive/BayArea500NoHighways/bad/out_output_451.png', '/content/drive/MyDrive/BayArea500NoHighways/val/output_453.png /content/drive/MyDrive/fixed_masks/output_1_fake_B_0.png /content/drive/MyDrive/BayArea500NoHighways/bad/out_output_453.png', '/content/drive/MyDrive/BayArea500NoHighways/val/output_452.png /content/drive/MyDrive/fixed_masks/output_2_fake_B_0.png /content/drive/MyDrive/BayArea500NoHighways/bad/out_output_452.png', '/content/drive/MyDrive/BayArea500NoHighways/val/output_454.png /content/drive/MyDrive/fixed_masks/output_3_fake_B_0.png /content/drive/MyDrive/BayArea500NoHighways/bad/out_output_454.png', '/content/drive/MyDrive/BayArea500NoHighways/val/output_456.png /content/drive/MyDrive/fixed_masks/output_4_fake_B_0.png /content/drive/MyDrive/BayArea500NoHighways/bad/out_output_456.png', '/content/drive/MyDrive/BayArea500NoHighways/val/

I use batch_test to generate the 54 real/fake pairs by modifying the checkpoint below. gan_mask is the model trained with the pix2pix masks

I draw on: https://torchmetrics.readthedocs.io/en/latest/references/modules.html#id8

In [ ]:
%cd /content/drive/MyDrive/generative_inpainting
!python /content/drive/MyDrive/generative_inpainting/batch_test.py --flist /content/test.flist --image_height 256 --image_width 256 --checkpoint_dir /content/drive/MyDrive/generative_inpainting/logs/gan_mask

I vectorize the 54 images

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
convert = transforms.ToTensor()
image_fake = Image.open("/content/drive/MyDrive/BayArea500NoHighways/bad/out_output_452.png")
t_fake = convert(image_fake)
t_fake = t_fake[0:3,:,:]
t_fake = torch.unsqueeze(t_fake,0)
image_real = Image.open("/content/drive/MyDrive/BayArea500NoHighways/val/output_452.png")
t_real = convert(image_real)
t_real = t_real[0:3,:,:]
t_real = torch.unsqueeze(t_real,0)
for i in range(451,504):
  image = Image.open("/content/drive/MyDrive/BayArea500NoHighways/bad/out_output_"+str(i)+".png")
  t1 = convert(image)
  t1 = t1[0:3,:,:]
  t1 = torch.unsqueeze(t1,0)
  t_fake = torch.cat((t_fake,t1),dim=0)
  image = Image.open("/content/drive/MyDrive/BayArea500NoHighways/val/output_"+str(i)+".png")
  t2 = convert(image)
  t2 = t2[0:3,:,:]
  t2 = torch.unsqueeze(t2,0)
  t_real = torch.cat((t_real,t2),dim=0)
print(t_real.size())
print(t_fake.size())

torch.Size([54, 3, 256, 256])
torch.Size([54, 3, 256, 256])


In [ ]:
#!pip install torchmetrics[image]
#!pip install lpips
from torchmetrics.image.lpip_similarity import LPIPS
lpips = LPIPS(net_type='vgg')
lpips(t_real, t_fake)

In [ ]:
from torchmetrics.image.inception import IS
_ = torch.manual_seed(3)
inception = IS()
t_fake = t_fake *255
t_fake = torch.tensor(t_fake,dtype=torch.uint8)
inception.update(t_fake)  
inception.compute() 

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `IS` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


(tensor(1.), tensor(6.2829e-08))

In [ ]:
from torchmetrics.image.fid import FID
fid = FID(feature=2048)
_ = torch.manual_seed(3)
t_real = t_real *255
t_real = torch.tensor(t_real,dtype=torch.uint8)
fid.update(t_real, real=True)  
fid.update(t_fake, real=False)  
fid.compute()  


/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `FID` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


tensor(54.4727)

I create the vectorized tensor to call IS scores on the test set and re-call cell above

In [ ]:
convert = transforms.ToTensor()
image_fake = Image.open("/content/drive/MyDrive/BayAreaTest/test/output_1_gen_gan.png")
t_fake = convert(image_fake)
t_fake = t_fake[0:3,:,:]
t_fake = torch.unsqueeze(t_fake,0)
for i in range(2,6):
  image = Image.open("/content/drive/MyDrive/BayAreaTest/test/output_"+str(i)+"_gen_gan.png")
  t1 = convert(image)
  t1 = t1[0:3,:,:]
  t1 = torch.unsqueeze(t1,0)
  t_fake = torch.cat((t_fake,t1),dim=0)
for i in range(1,6):
  image = Image.open("/content/drive/MyDrive/NOLATest/test/output_"+str(i)+"_gen_gan.png")
  t1 = convert(image)
  t1 = t1[0:3,:,:]
  t1 = torch.unsqueeze(t1,0)
  t_fake = torch.cat((t_fake,t1),dim=0)
t_fake = t_fake *255
t_fake = torch.tensor(t_fake,dtype=torch.uint8)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
